**Выполнил**

**Павлов Сергей Вячеславович**

**группа ДПИ22-1**

## Лабораторная работа №3



### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [165]:
import json
import pickle
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

with open('contributors_sample.json', 'rt', encoding='utf-8') as f:
    list_contributors = json.load(f)

s = []
for i in range(3):
    contributor = list_contributors[i]
    s2 =[]
    for details in contributor.keys():
        s2.append(f'{details}: {contributor[details]}')
    s.append('\n'.join(s2))
print('\n\n'.join(s))


username: uhebert
name: Lindsey Nguyen
sex: F
address: 01261 Cameron Spring
Taylorfurt, AK 97791
mail: jsalazar@gmail.com
jobs: ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer']
id: 35193

username: vickitaylor
name: Cheryl Lewis
sex: F
address: 66992 Welch Brooks
Marshallshire, ID 56004
mail: bhudson@gmail.com
jobs: ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial']
id: 91970

username: sheilaadams
name: Julia Allen
sex: F
address: Unit 1632 Box 2971
DPO AE 23297
mail: darren44@yahoo.com
jobs: ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile']
id: 1848091


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [166]:
unique_mail_domains = set()
for contributor in list_contributors:
    unique_mail_domains.add(contributor['mail'].split('@')[1])
len(unique_mail_domains)
for mail_domain in unique_mail_domains:
    print(mail_domain)

gmail.com
yahoo.com
hotmail.com


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [167]:
def find_username(username):
    try:
        for contributor in list_contributors:
            if username == contributor['username']:
                print('\n'.join([f'{detail}: {contributor[detail]}' for detail in contributor.keys()]))
                return
        raise ValueError("Пользователь '" + username + "' отсутствет.")
    except ValueError as e:
        print(str(e))

find_username('vickitaylor')
find_username('unknown_username')

username: vickitaylor
name: Cheryl Lewis
sex: F
address: 66992 Welch Brooks
Marshallshire, ID 56004
mail: bhudson@gmail.com
jobs: ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial']
id: 91970
Пользователь 'unknown_username' отсутствет.


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [168]:
amount_gender = {'F': 0, 'M': 0}
for contributor in list_contributors:
    amount_gender[contributor['sex']] += 1
print(f"Количество женщин: {amount_gender['F']}\nКоличество мужчин: {amount_gender['M']}")

Количество женщин: 2136
Количество мужчин: 2064


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [169]:
d = []
for contributor in list_contributors:
    d.append({'id': contributor['id'],'username': contributor['username'], 'sex': contributor['sex'] })

contributors = pd.DataFrame(d)
contributors

,id,username,sex
0,35193,uhebert,F
1,91970,vickitaylor,F
2,1848091,sheilaadams,F
3,50969,nicole82,F
4,676820,jean67,M
...,...,...,...
4195,423555,stevenspencer,F
4196,35251,rwilliams,M
4197,135887,lmartinez,F
4198,212714,brendahill,M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [170]:
recipes = pd.read_csv('recipes_sample.csv')
DataFrame2 = pd.merge(recipes, contributors, left_on='contributor_id', right_on='id', how='outer')

print(DataFrame2[DataFrame2['username'].isna()])

users = DataFrame2[DataFrame2['username'].isna()].groupby('contributor_id').count()
print("Информация отсутствует у ", len(users), " человек.")

                                     name    id_x  minutes  contributor_id  \
19           i can t believe it s spinach   38798       30            1533   
20                 3 corn casserole bread   79826       70            1533   
21                      abm spinach bread   19618       10            1533   
22       all canadian turkey cheeseburger   37313       31            1533   
23         almond crusted chicken breasts   22337       35            1533   
...                                   ...     ...      ...             ...   
29994                zucchini pizza boats  468144       35         1885554   
29995                     zucchini strips  279769       30          522304   
29996  zucchini with bacon corn   peppers  326105       45          896136   
29998           zucchini with serrano ham  162411       15          152500   
29999            zucchini potato pancakes  180833       45          186118   

        submitted  n_steps                                     

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [171]:
contributors_job = {}
for contributor in list_contributors:
    for job in contributor['jobs']:
        if job in contributors_job:
            contributors_job[job].append(contributor['username'])
        else:
            contributors_job[job] = [contributor['username']]
len(contributors_job)

639

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [172]:
with open('job_people.pickle', 'wb') as f:
    pickle.dump(contributors_job, f)

with open('job_people.json', 'wt', encoding='utf-8') as f:
    json.dump(contributors_job, f, indent=2)

if os.path.getsize('job_people.pickle') == os.path.getsize('job_people.json'):
    print("Объем файла 'job_people.pickle' равен объему файла 'job_people.json'")
elif os.path.getsize('job_people.pickle') < os.path.getsize('job_people.json'):
    print("Объем файла 'job_people.pickle' меньше объема файла 'job_people.json'")
else:
    print("Объем файла 'job_people.pickle' больше объема файла 'job_people.json'")

Объем файла 'job_people.pickle' меньше объема файла 'job_people.json'


2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [173]:
with open('job_people.pickle', 'rb') as f:
    demo = pickle.load(f)

#demo['Energy engineer']
if contributors_job == demo:
    print("Данные считались корректно.")

Данные считались корректно.


### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [174]:
with open('steps_sample.xml', 'rt') as f:
    content = BeautifulSoup(f, 'xml')

dict_recipes_steps = {}
for recipe in content.recipes.find_all('recipe'):
    dict_recipes_steps[recipe.find('id').text.strip()] = [step.text for step in recipe.steps.find_all('step')]

with open('steps_sample.json', 'wt', encoding='utf-8') as f:
    json.dump(dict_recipes_steps, f, indent=2)

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [175]:
dict_n_steps = {}

for recipe in content.recipes.find_all('recipe'):
    n_step = len([step for step in recipe.steps.find_all('step')])
    if n_step in dict_n_steps:
        dict_n_steps[n_step].append(recipe.find('id').text)
    else:
        dict_n_steps[n_step] = [recipe.find('id').text]

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [176]:
list_recipes_with_time = []

for recipe in content.recipes.find_all('recipe'):
    if len([x.attrs for x in recipe.steps.find_all('step') if x.attrs]) != 0:
        list_recipes_with_time.append(recipe.find('id').text)

#list_recipes_with_time

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [177]:
recipes = pd.read_csv('recipes_sample.csv')
steps ={k: len(v) for k,v in dict_recipes_steps.items()}
DataFrame2 = pd.DataFrame(steps.items(), columns=['id', 'n_steps'])
recipes[['id', 'n_steps']] = recipes[['id', 'n_steps']].fillna(DataFrame2)
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11.0,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3.0,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5.0,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,7.0,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,22.0,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,10.0,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,7.0,this is a delicious soup that i originally fou...,NaN


3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [178]:
n_steps_Nan = recipes['n_steps'].isna().sum()
print('Количество пропусков:', n_steps_Nan)

recipes.to_csv('recipes_sample_with_filled_nsteps.csv', index=False, encoding='utf-8')

Количество пропусков: 0
